In [1]:
import os
AVAILABLE_GPUS = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = AVAILABLE_GPUS

import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification, GPTNeoXForCausalLM

for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)

NVIDIA RTX A6000


In [2]:
# model_name_or_path = "OpenAssistant/pythia-12b-sft-v8-rlhf-2k-steps" #23GB
model_name_or_path = "OpenAssistant/pythia-12b-sft-v8-2.5k-steps" #23GB

model = GPTNeoXForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.float16, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side="left", legacy=False)
print(model.device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

cuda:0


In [3]:
import pandas as pd
import json

#only cut_off/5 different poems will be used
cut_off = 300
N_samples = 32
temperature = 0.8

def load_from_jsonl(file_name: str, cut_off: int) -> list[dict]:
    def load_json_line(line: str, i: int, file_name: str):
        try:
            return json.loads(line)
        except:
            raise ValueError(f"Error in line {i+1}\n{line} of {file_name}")
    with open(file_name, "r") as f:
        #only use the base, the like and the dislike prefix. And stop at cutof lines
        data = [load_json_line(line, i, file_name) for i, line in enumerate(f) if i<cut_off and i%5<3]
    return data

def get_prompt(d):
    prompt = d['prompt'][0]['content']
    OASS_prompt = '<|prompter|>' + prompt + '<|endoftext|>\n<|assistant|>Here are my comments:\n'
    return OASS_prompt 

dataset_filename: str = "feedback_poem.jsonl"
dataset_path = os.path.join("datasets", dataset_filename)
dataset = load_from_jsonl(dataset_path, cut_off)
print("Number of samples:",len(dataset))
prompts= [get_prompt(d) for d in dataset]

Number of samples: 180


In [4]:
def remove_the_excess_oes(prompt_array):
    array = []
    for el in prompt_array:
        while(el.count('<|endoftext|>')>2):
            el = el.rsplit('<|endoftext|>',1)[0]
        array.append(el)
    return array

In [5]:
batch_size = 8

output_array = []
#iteration over prompts with tqdm
for prompt in tqdm(prompts, desc= 'Poems', position=0):
    prompt_copy = [prompt]*N_samples
    temp_output_array = []
    for i in range(0, len(prompt_copy), batch_size):
        model_inputs = tokenizer(prompt_copy[i:i+batch_size], return_tensors="pt", padding=True).to("cuda")
        try: 
            generated_ids = model.generate(**model_inputs, max_length=800, pad_token_id=tokenizer.eos_token_id, eos_token_id= tokenizer.eos_token_id, do_sample=True, temperature=temperature)
        except:
            print("Cuda error")
            break
        output = tokenizer.batch_decode(generated_ids)
        temp_output_array.append(output)
    #flatten temp_output_array
    temp_output_array = [item for sublist in temp_output_array for item in sublist]
    temp_output_array = remove_the_excess_oes(temp_output_array)
    
    output_array.append(temp_output_array)
    
    # for i in range(5):
    #     #print(prompts[i])
    #     print(output[i].split('Here are my thoughts on the poem:')[1])
    #     print("#############################################")

Poems:  39%|███▉      | 70/180 [2:15:56<3:37:53, 118.85s/it]

In [ ]:
import datetime
now = datetime.datetime.now()

df = pd.DataFrame({'prompt': prompts, 'N_response': output_array})

df['type'] = df.index.map(lambda x: 'base' if x%3 == 0 else 'like' if x%3 == 1 else 'dislike')

#create a file name with current date and time
date_time = now.strftime("%m_%d_%Y_%H_%M_%S")
file_name = ("Bo"+ str(N_samples) +"_" + model_name_or_path + "_" + date_time + "_" + dataset_filename.split(".")[0] + "_" + str(cut_off) + ".csv").replace("/", "_")
file_path = "results/" + file_name
print("file name:", file_name)

#save dataframe as csv
df.to_csv(file_path, index=False)

df.head()


file name: Bo32_OpenAssistant_pythia-12b-sft-v8-2.5k-steps_11_16_2023_15_10_37_feedback_poem_5.csv


,prompt,N_response,type
0,<|prompter|>Please comment briefly on the foll...,[<|prompter|>Please comment briefly on the fol...,base
1,<|prompter|>Please comment briefly on the foll...,[<|prompter|>Please comment briefly on the fol...,like
2,<|prompter|>Please comment briefly on the foll...,[<|prompter|>Please comment briefly on the fol...,dislike
